# **Criando o Banco de Dados**

Notebook com os comandos necessários para criar um banco SQL no SQLite e carregar os dados dos arquivos fornecidos nele.

**Arquivos fornecidos:**
- *train_houses.xlsx*
- *test_houses.xlsx*

In [14]:
import pandas as pd
from sqlalchemy import create_engine, text

In [15]:
# Função para carregar XLSX para o banco de dados
def carregar_xlsx_para_sqlite(xlsx_path, sheet_name, table_name):
    # Ler o XLSX
    df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
    
    # Carregar dados para o banco de dados
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Tabela {table_name} criada e dados carregados.")

In [16]:
# Definindo nome do banco
db_name = 'data/HousePrices.db'
engine = create_engine(f'sqlite:///{db_name}')

In [17]:
# Especifique os arquivos XLSX e as tabelas correspondentes
arquivos_xlsx = {
    'data/train_houses.xlsx': ('in', 'train_houses'),
    'data/test_houses.xlsx': ('in', 'test_houses')
}

In [18]:
# Carregar todos os XLSXs para o banco de dados
for xlsx_path, (sheet_name, table_name) in arquivos_xlsx.items():
    carregar_xlsx_para_sqlite(xlsx_path, sheet_name, table_name)

Tabela train_houses criada e dados carregados.
Tabela test_houses criada e dados carregados.


In [19]:
# Verificar se os dados foram carregados corretamente
with engine.connect() as conn:
    for _, (_, table_name) in arquivos_xlsx.items():
        result = conn.execute(text(f"SELECT * FROM {table_name} LIMIT 5")).fetchall()
        print(f"Dados da {table_name}:")
        for row in result:
            print(row)

Dados da train_houses:
('Pre-owned Condominiums, etc.,,13103,Tokyo,Minato Ward,Kaigan,Takeshiba,1,1.0,1.0,1K,35,0,,,,,False,,0,1979.0,0,SRC,House,,,,,Quasi-industrial Zone,60.0,400.0,1st quarter 2011,2011,1,Done,,24000000',)
('Residential Land(Land and Building),Residential Area,13120,Tokyo,Nerima Ward,Nishiki,Kamiitabashi,15,15.0,15.0,,110,0,,,Semi-rectangular Shaped,12.0, ... (31 characters truncated) ... ,East,Private Road,4.0,Category I Exclusively Low-story Residential Zone,60.0,200.0,3rd quarter 2013,2013,3,,Dealings including private road,51000000',)
('Residential Land(Land Only),Residential Area,13201,Tokyo,Hachioji City,Shimoongatamachi,Takao (Tokyo),1H-1H30,60.0,90.0,,135,0,100000.0,340000.0,Irregular Shaped,10.0,False,,0,,0,,,,East,Private Road,4.5,Category I Exclusively Low-story Residential Zone,40.0,80.0,4th quarter 2007,2007,4,,,14000000',)
('Pre-owned Condominiums, etc.,,13208,Tokyo,Chofu City,Kamiishiwara,Nishichofu,16,16.0,16.0,3LDK,60,0,,,,,False,,0,1999.0,0,RC,House